All about union and unionAll
-----------------------------
1. Difference beween union and unionAll?
2. What will happen if I change the no. of columns while unioning the data?
3. What if column name is different?
4. What is unionByName?


###### Dataframe union()
union() method of the DataFrame is used to combine two DataFrame’s of the same structure/schema.<br>
 If schemas are not the same it returns an error.

###### DataFrame unionAll()
unionAll() is deprecated since Spark “2.0.0” version and replaced with union().

Note: In other SQL’s, Union eliminates the duplicates but UnionAll combines two datasets including duplicate records.<br>
But, in spark both behave the same and use [DataFrame duplicate function to remove duplicate rows.](https://stackoverflow.com/questions/29022530/why-would-i-want-union-over-unionall-in-spark-for-schemardds)

###### Bonus fact from spark documentation : 
In Spark 3.0, the Dataset and DataFrame API unionAll is no longer deprecated. It is an alias for union.


In [0]:
data = [(10, 'Anil', 50000, 18),
        (11, 'Vikas', 75000, 16),
        (12, 'Nisha', 40000, 18),
        (13, 'Nidhi', 60000, 17),
        (14, 'Priya', 80000, 18),
        (15, 'Mohit', 45000, 18),
        (16, 'Rajesh', 90000, 10),
        (17, 'Raman', 55000, 16),
        (18, 'Sam', 65000,   17)]

# Defining schema
schema = ['id', 'name', 'sal', 'mgr_id']

# Creating dataframe
manager_df = spark.createDataFrame(data=data, schema=schema)

# Show the dataframe
manager_df.show()

+---+------+-----+------+
| id|  name|  sal|mgr_id|
+---+------+-----+------+
| 10|  Anil|50000|    18|
| 11| Vikas|75000|    16|
| 12| Nisha|40000|    18|
| 13| Nidhi|60000|    17|
| 14| Priya|80000|    18|
| 15| Mohit|45000|    18|
| 16|Rajesh|90000|    10|
| 17| Raman|55000|    16|
| 18|   Sam|65000|    17|
+---+------+-----+------+



In [0]:
# Count the number of record in dataframe           
manager_df.count()

Out[14]: 9

In [0]:
data1=[(19 ,'Sohan',50000, 18),
(20 ,'Sima',75000,  17)]

# Defining schema
schema1 = ['id', 'name', 'sal', 'mgr_id']

# Creating dataframe
manager_df1 = spark.createDataFrame(data = data1,schema = schema1)

# Show the dataframe
manager_df1.show()

+---+-----+-----+------+
| id| name|  sal|mgr_id|
+---+-----+-----+------+
| 19|Sohan|50000|    18|
| 20| Sima|75000|    17|
+---+-----+-----+------+



In [0]:
# Apply union transformation on both the dataframe
manager_df.union(manager_df1).show()

+---+------+-----+------+
| id|  name|  sal|mgr_id|
+---+------+-----+------+
| 10|  Anil|50000|    18|
| 11| Vikas|75000|    16|
| 12| Nisha|40000|    18|
| 13| Nidhi|60000|    17|
| 14| Priya|80000|    18|
| 15| Mohit|45000|    18|
| 16|Rajesh|90000|    10|
| 17| Raman|55000|    16|
| 18|   Sam|65000|    17|
| 19| Sohan|50000|    18|
| 20|  Sima|75000|    17|
+---+------+-----+------+



In [0]:
# Counting number of records after unioning both the dataframe
manager_df.union(manager_df1).count()

Out[5]: 11

In [0]:
data2 = [(10, 'Anil', 50000, 18),
        (11, 'Vikas', 75000, 16),
        (12, 'Nisha', 40000, 18),
        (13, 'Nidhi', 60000, 17),
        (14, 'Priya', 80000, 18),
        (15, 'Mohit', 45000, 18),
        (16, 'Rajesh', 90000, 10),
        (17, 'Raman', 55000, 16),
        (18, 'Sam', 65000,   17),
        (18, 'Sam', 55000,   17),
        (18, 'Sam', 65000,   17)]

# Defining schema
schema2 = ['id', 'name', 'sal', 'mgr_id']

# Creating duplicate_manager_df1 dataframe by keeping some duplicate value of manager_df1
duplicate_manager_df1 = spark.createDataFrame(data=data2, schema=schema2)


In [0]:
# Let's try to apply both union and unionAll on manager_df1 and duplicate_manager_df1
duplicate_manager_df1.union(manager_df1).show()

+---+------+-----+------+
| id|  name|  sal|mgr_id|
+---+------+-----+------+
| 10|  Anil|50000|    18|
| 11| Vikas|75000|    16|
| 12| Nisha|40000|    18|
| 13| Nidhi|60000|    17|
| 14| Priya|80000|    18|
| 15| Mohit|45000|    18|
| 16|Rajesh|90000|    10|
| 17| Raman|55000|    16|
| 18|   Sam|65000|    17|
| 18|   Sam|55000|    17|
| 18|   Sam|65000|    17|
| 19| Sohan|50000|    18|
| 20|  Sima|75000|    17|
+---+------+-----+------+



In [0]:
# Count total record after applying union on duplicate dataframe
duplicate_manager_df1.union(manager_df1).count()

Out[35]: 13

In [0]:
# Count total record after applying unionAll on duplicate dataframe
duplicate_manager_df1.unionAll(manager_df1).count()

Out[36]: 13

In [0]:
# As demonstrated above, despite employing duplicate values, both the `union` and `unionAll` operations 
# yield all records from both dataframes. This observation highlights that in Spark, both operations exhibit similar 
# behavior, as previously mentioned.

In [0]:
# Let's do the same operation in sparkSQL
# Before that we have to convert it to temporary view or table
manager_df1.createOrReplaceTempView("manager_tbl")
duplicate_manager_df1.createOrReplaceTempView("duplicate_manager_tbl")

In [0]:
# As you can see in SparkSQL, the union merged two table and removed duplicate(in dataframe it was showing 13 records)
spark.sql("""
         SELECT COUNT(*) AS total_record_after_union
         FROM(
          SELECT * FROM manager_tbl
          UNION
          SELECT * FROM duplicate_manager_tbl
         )
          """).show()

+------------------------+
|total_record_after_union|
+------------------------+
|                      12|
+------------------------+



In [0]:
# As you can see in SparkSQL, the uniona ll merged two tables but didn't remove duplicate(in dataframe it was showing 13 records)
spark.sql("""
         SELECT COUNT(*) AS total_record_after_union_all
         FROM(
          SELECT * FROM manager_tbl
          UNION ALL
          SELECT * FROM duplicate_manager_tbl
         )
          """).show()

+----------------------------+
|total_record_after_union_all|
+----------------------------+
|                          13|
+----------------------------+



In [0]:
spark.sql("""
          SELECT * FROM duplicate_manager_tbl
          UNION
          SELECT * FROM duplicate_manager_tbl
          """).count()

Out[43]: 10


##### My understanding on above query
The SparkSQL code produced 10 unique records by using UNION on both views. The 'duplicate_manager_tbl'<br>
originally had 11 records, including one duplicate. The UNION operation combined the tables, resulting in 22 records,<br>
and then removed the two duplicates, leaving a final output of 10 records.

[Where did I get to know this](https://discuss.codecademy.com/t/what-happens-if-tables-we-perform-union-on-have-duplicate-rows/356107)
###### Some additional resources to understand these topics better

[StrataScratch Blog](https://www.stratascratch.com/blog/sql-union-vs-union-all-differences-you-need-to-know/#:~:text=One%20important%20feature%20of%20UNION,UNION%20to%20remove%20that%20row.)

[Chat with Google Bard](https://g.co/bard/share/a44e01018f49)


In [0]:
wrong_column_data=[(19 ,50000, 18,'Sohan'),
(20 ,75000,  17,'Sima')]

schema3 = ['id','sal','mgr_id','name']
wrong_manager_df = spark.createDataFrame(data = wrong_column_data,schema = schema3)
wrong_manager_df.show()

+---+-----+------+-----+
| id|  sal|mgr_id| name|
+---+-----+------+-----+
| 19|50000|    18|Sohan|
| 20|75000|    17| Sima|
+---+-----+------+-----+



In [0]:
# Let's see what happens we change the order of the column
# Let's use union on wrong_manager_df and manager_df1
wrong_manager_df.union(manager_df1).show()
# As you can see Spark places the record as is, without knowledge of the positioning for the 'id' and 'name' 
# values in corresponding column.

+---+-----+------+-----+
| id|  sal|mgr_id| name|
+---+-----+------+-----+
| 19|50000|    18|Sohan|
| 20|75000|    17| Sima|
| 19|Sohan| 50000|   18|
| 20| Sima| 75000|   17|
+---+-----+------+-----+



unionByName
-------------
```To resolve the above issue spark provides another method i.e unionByName```<br>

The unionByName() function in PySpark is used to union two DataFrames when you have column names in a different order or<br>
even if you have missing columns in any DataFrame, in other words, this function resolves columns by name (not by position).

Syntax : ```DataFrame.unionByName(other, allowMissingColumns=False)```

The allowMissingColumns parameter is a boolean value that specifies whether or not to allow missing columns in<br>
the other DataFrame. If this parameter is set to True, then any missing columns in the other DataFrame will be added<br>
to the current DataFrame with a default value of null.

In [0]:
# As you can see here it resolves the above issue by placing the values in corresponding column
wrong_manager_df.unionByName(manager_df1).show()

+---+-----+------+-----+
| id|  sal|mgr_id| name|
+---+-----+------+-----+
| 19|50000|    18|Sohan|
| 20|75000|    17| Sima|
| 19|50000|    18|Sohan|
| 20|75000|    17| Sima|
+---+-----+------+-----+



###### Limitation of unionByName
But it has some limitation i.e it cannot be used to combine DataFrames with different schemas. If the two DataFrames<br>
have different columns, or if the columns have different data types, the unionByName() function will fail.


In [0]:
# Creating new dataframe with updated schema to demonstrate the limitation of unionByName
wrong_column_data1=[(19 ,50000, 18,'Sohan'),
(20 ,75000,  17,'Sima')]

schema4 = ['id','sal','mgr_id','nam']
updated_schema_wrong_manager_df = spark.createDataFrame(data = wrong_column_data1,schema = schema4)
updated_schema_wrong_manager_df.show()

+---+-----+------+-----+
| id|  sal|mgr_id|  nam|
+---+-----+------+-----+
| 19|50000|    18|Sohan|
| 20|75000|    17| Sima|
+---+-----+------+-----+



In [0]:
# As you can see here it is throwing an Analysis exception because both the dataframe have different schema
updated_schema_wrong_manager_df.unionByName(manager_df1).show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2310222119595342>:2
      1 # As you can see here it is throwing an exception
----> 2 updated_schema_wrong_manager_df.unionByName(manager_df1).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3738, in DataFrame.unionByName(self, other, allowMissingColumns)
   3682 def unionByName(self, other: "DataFrame", allowMissingColumns: bool = False) -> "DataFrame":
   3683     """Returns a new :class:`DataFrame` containing union of rows in this and anot

In [0]:
# Creating a dataframe with extra column named bonus
data3 = [(10, 'Anil', 50000, 18,20),
               (11, 'Vikas', 75000, 16,32),
               (12, 'Nisha', 40000, 18,35),
               (13, 'Nidhi', 60000, 17,15)]
schema6 = ['id','name','sal','mgr_id','bonus']
manager_df2 = spark.createDataFrame(data = data3,schema = schema6)

In [0]:
# Let's check what happens if we change the number of columns while applying union
manager_df1.union(manager_df2).show()
# As you can clearly see here it is throwing Analysis Exception saying mismatch of no. of columns.
# So we have to always apply union on dataframes with same no. of columns

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2310222119595345>:2
      1 # Let's check what happens if we change the number of columns while applying union
----> 2 manager_df1.union(manager_df2).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3646, in DataFrame.union(self, other)
   3598 def union(self, other: "DataFrame") -> "DataFrame":
   3599     """Return a new :class:`DataFrame` containing union of rows in this and another
   3600     :class:`DataFrame`.
   3601 
   (...)
   3644 

In [0]:
# We can resolve the above issue using unionByName but we have to set True in allowMissingColumns parameter
manager_df1.unionByName(manager_df2,allowMissingColumns=True).show()
# Note: As per the video we can do it without using unionByName i.e by selecting only common column
# i.e manager_df1.select('id','name','sal','mgr_id').union(manager_df2).show()
# But don't know why it is giving the same error as above as column mismatch. I will check it later.

+---+-----+-----+------+-----+
| id| name|  sal|mgr_id|bonus|
+---+-----+-----+------+-----+
| 19|Sohan|50000|    18| null|
| 20| Sima|75000|    17| null|
| 10| Anil|50000|    18|   20|
| 11|Vikas|75000|    16|   32|
| 12|Nisha|40000|    18|   35|
| 13|Nidhi|60000|    17|   15|
+---+-----+-----+------+-----+



In [0]:
# Creating a dataframe with extra column named bonus and changing the order of the columns
wrong_column_data3=[(19 ,50000, 18,'Sohan',10),
(20 ,75000,  17,'Sima',20)]
schema6 = ['id','sal','mgr_id','name','bonus']

wrong_manager_df2 = spark.createDataFrame(data = wrong_column_data3,schema = schema6)
wrong_manager_df2.show()


+---+-----+------+-----+-----+
| id|  sal|mgr_id| name|bonus|
+---+-----+------+-----+-----+
| 19|50000|    18|Sohan|   10|
| 20|75000|    17| Sima|   20|
+---+-----+------+-----+-----+



In [0]:
# Same thing was not working with manager_df1 and manager_df2
# But as we can see this is working here means although wrong_manager_df2 has one more additional
# column than manager_df1 but by selecting only common column we can combine both the dataframe
# using union. Since the order of the columns are different in both the df for which it is not showing
# corresponding value in respective column but we can fix this using unionByName as we have already
# done earlier.
# Note: I will try to figure out why same thing was not working on manager_df1 and manager_df2
wrong_manager_df2.select('id','sal','mgr_id','name').union(manager_df1).show()

+---+-----+------+-----+
| id|  sal|mgr_id| name|
+---+-----+------+-----+
| 19|50000|    18|Sohan|
| 20|75000|    17| Sima|
| 19|Sohan| 50000|   18|
| 20| Sima| 75000|   17|
+---+-----+------+-----+

